# Data Modelling

**Building of the initial dataset, which is used in PandaPower later.**

- Every time series ist normed to 1000 kWh/a 
- Due to the leap year in 2020 some time series, which were not created for 2020, are not normed to 1000 

In [1]:
import os
import arrow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cwd = os.getcwd()
print(cwd)
profiles_fn = os.path.join(cwd, 'data_modelling', 'profiles')
print(profiles_fn)

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator
/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling/profiles


## Build Index depending on duration

In [3]:
start_ts = arrow.get('01.01.2020')
duration_years = 4
duration = 365 * duration_years

In [4]:
print(start_ts)

2020-01-01T00:00:00+00:00


In [5]:
end_ts = start_ts.shift(days=duration, hours=23, minutes=45)

In [6]:
print(end_ts)

2023-12-31T23:45:00+00:00


In [7]:
fmt = 'YYYY-MM-DD HH:mm:ss' # index date has to be in the correct format for pandapower
current_ts = start_ts
ts_list = []
ts_list.append(current_ts.format(fmt))

while current_ts < end_ts:
    current_ts = current_ts.shift(minutes=15)
    ts_list.append(current_ts.format(fmt))

In [8]:
ts_list[201]

'2020-01-03 02:15:00'

In [9]:
current_day = start_ts
end_day = start_ts.shift(days=duration)
days_list = []
days_list.append(start_ts)

while current_day < end_day:
    current_day = current_day.shift(days=1)
    days_list.append(current_day)

In [10]:
current_month = start_ts
month_list = []
month_list.append(current_month.month)

while current_month <= end_day:
    current_month = current_month.shift(months=1)
    month_list.append(current_month.month)

In [11]:
month_list = month_list[:-1] # delete last item since its not neccessary

In [12]:
winter = [12, 1, 2]
sommer = [6, 7, 8]
uebergang = [3, 4, 5, 9, 10, 11]

## Read profiles

In [13]:
df_h0_profiles = pd.read_csv(os.path.join(profiles_fn, 'H0_profile.csv'),sep=';', dtype=np.float64, decimal=',')

In [14]:
df_l0_profiles = pd.read_csv(os.path.join(profiles_fn, 'L0_profile.csv'),sep=';', dtype=np.float64, decimal=',')

In [15]:
df_g0_profiles = pd.read_csv(os.path.join(profiles_fn, 'G0_profile.csv'),sep=';', dtype=np.float64, decimal=',')

In [16]:
df_pv_profiles = pd.read_csv(os.path.join(profiles_fn, 'PV_profile.csv'),sep=';', dtype=np.float64, decimal=',')

In [17]:
df_wind_data = pd.read_csv(os.path.join(profiles_fn, 'WND_profile.csv'), sep=';', dtype=np.float64, decimal=',')

In [18]:
df_gas_data = pd.read_csv(os.path.join(profiles_fn, 'gas_profile.csv'), sep=';', dtype=np.float64, decimal=',')

## Build Dataset

In [19]:
def build_tag(profile, month, day):
    tmplt = '{}_{}_{}'
     
    if month in winter:
        season_txt = 'winter'
    elif month in sommer:
        season_txt = 'sommer'
    elif month in uebergang:
        season_txt = 'uebergang'
        
    if day == 6:
        day_txt = 'sa'
    elif day == 7:
        day_txt = 'so'
    else:
        day_txt = 'wt'
    
    return tmplt.format(profile, season_txt, day_txt)

In [20]:
def build_tag_pv(month, day):
    tmplt = "pv_{}_{}"
    
    if month == 1:
        month_txt = 'jan'
        season_txt = 'winter'
    elif month == 2:
        month_txt = 'feb'
        season_txt = 'winter'
    elif month == 3:
        month_txt = 'mrz'
        season_txt = 'winter'
    elif month == 4:
        month_txt = 'apr'
        season_txt = 'sommer'
    elif month == 5:
        month_txt = 'mai'
        season_txt = 'sommer'
    elif month == 6:
        month_txt = 'jun'
        season_txt = 'sommer'
    elif month == 7:
        month_txt = 'jul'
        season_txt = 'sommer'
    elif month == 8:
        month_txt = 'aug'
        season_txt = 'sommer'
    elif month == 9:
        month_txt = 'sep'
        season_txt = 'sommer'
    elif month == 10:
        month_txt = 'okt'
        season_txt = 'sommer'
    elif month == 11:
        month_txt = 'nov'
        season_txt = 'sommer'
    elif month == 12:
        month_txt = 'dez'
        season_txt = 'sommer'
        
    return tmplt.format(season_txt, month_txt)

In [21]:
def build_tag_wind(year, month):
    tmplt = 'wind_{}'
    leap_years = [2020, 2024, 2028, 2032]
    
    if year in leap_years and month == 2:
        return 'wind_feb_leap_y'
    
    if month == 1:
        month_txt = 'jan'
    elif month == 2:
        month_txt = 'feb'
    elif month == 3:
        month_txt = 'mrz'
    elif month == 4:
        month_txt = 'apr'
    elif month == 5:
        month_txt = 'mai'
    elif month == 6:
        month_txt = 'jun'
    elif month == 7:
        month_txt = 'jul'
    elif month == 8:
        month_txt = 'aug'
    elif month == 9:
        month_txt = 'sep'
    elif month == 10:
        month_txt = 'okt'
    elif month == 11:
        month_txt = 'nov'
    elif month == 12:
        month_txt = 'dez'
    
    return tmplt.format(month_txt)

In [22]:
def select_wind(year, month):
    
    select_tag = build_tag_wind(year, month)
    wnd_df = df_wind_data[select_tag]
    
    wnd_df.dropna(inplace=True)

    return wnd_df

In [23]:
def build_tag_gas(year, month):
    tmplt = 'gas_{}'
    leap_years = [2020, 2024, 2028, 2032]
    
    if year in leap_years and month == 2:
        return 'gas_feb_leap_y'
    
    if month == 1:
        month_txt = 'jan'
    elif month == 2:
        month_txt = 'feb'
    elif month == 3:
        month_txt = 'mrz'
    elif month == 4:
        month_txt = 'apr'
    elif month == 5:
        month_txt = 'mai'
    elif month == 6:
        month_txt = 'jun'
    elif month == 7:
        month_txt = 'jul'
    elif month == 8:
        month_txt = 'aug'
    elif month == 9:
        month_txt = 'sep'
    elif month == 10:
        month_txt = 'okt'
    elif month == 11:
        month_txt = 'nov'
    elif month == 12:
        month_txt = 'dez'
    
    return tmplt.format(month_txt)

In [24]:
def select_gas(year, month):
    
    select_tag = build_tag_gas(year, month)
    print(select_tag)
    gas_df = df_gas_data[select_tag]
    
    gas_df.dropna(inplace=True)

    return gas_df

## Append Data per Day

In [25]:
h0_data = []
g0_data = []
l0_data = []
pv_data = []

for day in days_list:
    day_of_week = day.isoweekday()
    month = day.month
    year = day.year
    
    h0_select = build_tag('h0', month, day_of_week)
    g0_select = build_tag('g0', month, day_of_week)
    l0_select = build_tag('l0', month, day_of_week)
    pv_select = build_tag_pv(month, day_of_week)
    
    h0_tmp = df_h0_profiles[h0_select]
    g0_tmp = df_g0_profiles[g0_select]
    l0_tmp = df_l0_profiles[l0_select]
    pv_tmp = df_pv_profiles[pv_select]
    
    for item in h0_tmp:
        h0_data.append(item)
    
    for item in g0_tmp:
        g0_data.append(item)
        
    for item in l0_tmp:
        l0_data.append(item)
        
    for item in pv_tmp:
        pv_data.append(item)
        


### Append Data per Month

In [26]:
wnd_data = []
gas_data = []

current_year = 2020
last_month = 0

for month in month_list:

    if last_month == 12 and month == 1:
        current_year += 1
    
    wnd_tmp = select_wind(current_year, month)
    gas_tmp = select_gas(current_year, month)
    
    last_month = month
    
    for item in wnd_tmp:
        wnd_data.append(item)
        
    for item in gas_tmp:
        gas_data.append(item)
    

gas_jan
gas_feb_leap_y
gas_mrz
gas_apr
gas_mai
gas_jun
gas_jul
gas_aug
gas_sep
gas_okt
gas_nov
gas_dez
gas_jan
gas_feb
gas_mrz
gas_apr
gas_mai
gas_jun
gas_jul
gas_aug
gas_sep
gas_okt
gas_nov
gas_dez
gas_jan
gas_feb
gas_mrz
gas_apr
gas_mai
gas_jun
gas_jul
gas_aug
gas_sep
gas_okt
gas_nov
gas_dez
gas_jan
gas_feb
gas_mrz
gas_apr
gas_mai
gas_jun
gas_jul
gas_aug
gas_sep
gas_okt
gas_nov
gas_dez


In [27]:
## Build initial DataFrame
df_data = pd.DataFrame(index=ts_list)

In [28]:
## Negate every genereration
pv_data = [x * -1 for x in pv_data]
wind_data = [x * -1 for x in wnd_data]
gas_data_list = [x * -1 for x in gas_data]

In [29]:
df_data['load_h0_normed_kW'] = h0_data
df_data['load_g0_normed_kW'] = g0_data
df_data['load_l0_normed_kW'] = l0_data
df_data['gen_pv_normed_kW'] = pv_data
df_data['gen_wind_normed_kW'] = wind_data
df_data['gen_gas_normed_kW'] = gas_data_list

In [30]:
df_data.head(10)

,load_h0_normed_kW,load_g0_normed_kW,load_l0_normed_kW,gen_pv_normed_kW,gen_wind_normed_kW,gen_gas_normed_kW
2020-01-01 00:00:00,67.6,65.5,72.4,-0.0,-0.167,-0.058531
2020-01-01 00:15:00,60.8,62.6,69.4,-0.0,-0.166,-0.058531
2020-01-01 00:30:00,54.9,59.6,66.9,-0.0,-0.165,-0.058531
2020-01-01 00:45:00,49.9,57.0,64.8,-0.0,-0.164,-0.058531
2020-01-01 01:00:00,46.2,54.8,63.2,-0.0,-0.163,-0.058531
2020-01-01 01:15:00,43.6,53.1,61.9,-0.0,-0.162,-0.058531
2020-01-01 01:30:00,41.9,51.7,61.0,-0.0,-0.162,-0.058531
2020-01-01 01:45:00,40.8,50.5,60.2,-0.0,-0.161,-0.058531
2020-01-01 02:00:00,40.1,49.4,59.6,-0.0,-0.160,-0.058531
2020-01-01 02:15:00,39.6,48.5,59.2,-0.0,-0.159,-0.058531


In [31]:
df_data.to_csv(os.path.join(cwd, 'data_modelling', 'main_profiles.csv'),sep=';', index_label='index')